# Install

In [ ]:
!pip3 install soynlp

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install swifter

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [ ]:
import sys
sys.path.append('drive/MyDrive/MemeProject/src/doc2vec/')
print(sys.path)

['/content', '/env/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.8/dist-packages/IPython/extensions', '/root/.ipython', 'drive/MyDrive/MemeProject/src/doc2vec/']


In [ ]:
import re
import config as cfg
import swifter
from pshmodule.utils import filemanager as fm

In [ ]:
df = fm.load(cfg.temp_data_colab)

extension : .pickle
Loaded 9141 records from drive/MyDrive/MemeProject/data/temp_for_doc2vec.pickle


In [ ]:
df.head()

,u,soynlp
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반에, 서, 1등, 했어]"
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내가, 반에, 서, 제일, 잘, 받음]"
2,나 반에서 중간 성적 제일 좋아,"[나, 반에, 서, 중간, 성적, 제일, 좋아]"
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반에, 서, 내가, 시험, 제일, 잘, 봤다]"
4,중간고사 반 1등 먹음,"[중간고사, 반, 1등, 먹음]"


In [ ]:
df_ref = fm.load(cfg.origin_ref)

extension : .pickle
Loaded 181704 records from drive/MyDrive/MemeProject/data/origin_ref.pickle


In [ ]:
df_ref.head()

,num,g_num,u_num,u,meme,arg1,arg_op1,arg_op2
0,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 것도 곁들인...,중간고사,,
1,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 내 몫을 제일 많이 곁들인...,중간고사,,
2,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 대춘이 몫도 곁들인...,중간고사,,
3,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 배보다 배꼽이 더 큰 걸 곁들인...,중간고사,,
4,1,1,1,나 중간고사 반에서 1등했어,그렇다면 통 크게 함 쏴야지! 근데 이제 과소비를 곁들인...,중간고사,,


In [ ]:
# df_ref['u'] = df_ref.u.swifter.apply(lambda x: re.sub(r'[^\uAC00-\uD7A30-9a-zA-Z\s]', '', x))

# Model Load

In [ ]:
# from konlpy.tag import Okt
from gensim.models.doc2vec import Doc2Vec

In [ ]:
# okt = Okt()
# model_okt = Doc2Vec.load(cfg.doc2vec_okt_colab)

In [ ]:
model_soynlp = Doc2Vec.load(cfg.doc2vec_soynlp_colab)

# Memebot with doc2vec Similarity

### 학습된 문장 유사도

In [ ]:
input_sent = input()

오늘 날씨가 너무 좋다


##### okt

In [ ]:
index = 4
print(df.content.iloc[index])
print("-"*100)
result = model_okt.docvecs.most_similar(index)
for k, v in enumerate(result):
    if k == 0:
        print(f"k : {k}")
        print(f"v : {v}")
        print(df.content.iloc[int(v[0])])

중간고사 반 1등 먹음
----------------------------------------------------------------------------------------------------
k : 0
v : ('0', 0.915203332901001)
나 중간고사 반에서 1등했어


In [ ]:
df_ref.tail()

,u,cpr,mm,cpo
208695,전국노래자랑에서 이모가 1등했다,대길이 이모님... 힘숨찐ㄷㄷ,상상도 못한 정체 ㄴㅇㄱ,
208696,전국노래자랑에서 이모가 1등했다,대길이 이모님 힘숨찐이시네!,상상도 못한 정체 ㄴㅇㄱ,
208697,전국노래자랑에서 이모가 1등했다,대길이 이모님 힘숨찐이시라능,상상도 못한 정체 ㄴㅇㄱ,
208698,전국노래자랑에서 이모가 1등했다,대길좌 이모님이 우승자라구?!,상상도 못한 정체 ㄴㅇㄱ,
208699,전국노래자랑에서 이모가 1등했다,"대길쓰 이모님,, 힘숨찐ㄷㄷ",상상도 못한 정체 ㄴㅇㄱ,


입력 문장 유사도

In [ ]:
# vectorize new sentences based on doc2vec
input_vec = model_okt.infer_vector(okt.morphs(input_sent))

# most similarity top 5
return_docs = model_okt.docvecs.most_similar(positive=[input_vec], topn=5)

# find Similar Sentences in the Trained Dataframe
result = [df.content.iloc[int(i[0])] for i in return_docs]

# result
if df_ref.empty:
    print("유사 문장이 없습니다.")
else:
    print(f"입력 문장 : {input_sent}")
    print("-"*100)
    for k, i in enumerate(zip(return_docs, result)):
        print(f"{k+1}.")
        print(f"유사 문장 : {i[1]}")
        print(f"유사 문장 : {i[0][1]}")
        # find Similar Sentences in Dialogueset
        temp_ref = df_ref[df_ref.u == i[1]]
        temp_ref = temp_ref.sample(frac=1).reset_index(drop=True)
        result = str(temp_ref.cpr.iloc[0]) + " " + str(temp_ref.mm.iloc[0]) + " " + str(temp_ref.cpo.iloc[0])
        print(f"밈 답변 : {result.strip()}")
        print("-"*100)

입력 문장 : 화나게 하지 말고 저리 꺼져
----------------------------------------------------------------------------------------------------
1.
유사 문장 : 이 녀석 꽤 센스좋네
유사 문장 : 0.6098244786262512
밈 답변 : 알잘딱깔센!! 그게 바로 나☆
----------------------------------------------------------------------------------------------------
2.
유사 문장 : 네가 회사로 픽업 오라니까
유사 문장 : 0.606505274772644
밈 답변 : 왜요? 제가 대길이 전용인 기사처럼 보이나요? 얼마 줄 건지 딱 말해
----------------------------------------------------------------------------------------------------
3.
유사 문장 : 저기서 술 좀 건네줘 대춘쓰
유사 문장 : 0.5976459383964539
밈 답변 : 왜요? 제가 시키는 대로 순순히 하는 대춘이처럼 생겼나요? 말을 잇지 못 하는...
----------------------------------------------------------------------------------------------------
4.
유사 문장 : 나 요즘 간식을 꾸준히 입에 집어넣음
유사 문장 : 0.5883457660675049
밈 답변 : 야 너두?! 야 나두!!
----------------------------------------------------------------------------------------------------
5.
유사 문장 : 니 막말로 무쓸모네
유사 문장 : 0.5842003226280212
밈 답변 : 아뉜데 아뉜데 반박은 현금으로 받겠습니다
--------------------------------

##### soynlp

In [ ]:
index = 0
print(df.u.iloc[index])
print("-"*100)
result = model_soynlp.docvecs.most_similar(index)

for k, v in enumerate(result):
    if k == 0:
        print(f"k : {k}")
        print(f"v : {v}")
        print(df.u.iloc[int(v[0])])

u    나 중간고사 반에서 1등했어
Name: 0, dtype: object
----------------------------------------------------------------------------------------------------
k : 0
v : ('3550', 0.8649837970733643)
u    나 반에서 인기 투표 1위했어
Name: 3550, dtype: object


입력 문장 유사도

In [ ]:
from soynlp.word import WordExtractor
from soynlp.tokenizer import LTokenizer

ltokenizer setting

In [ ]:
# WordExtractor
word_extractor = WordExtractor(
    min_frequency=10,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)

word_extractor.load(cfg.soynlp)
words = word_extractor.extract()

cohesion_score = {word:score.cohesion_forward for word, score in words.items()}

# 사용자 사전 추가
with open(cfg.user_dict, 'r') as f:
  user_dict = f.readlines()

user_dict = [i.replace('\n', '') for i in user_dict]

for i in user_dict:
  cohesion_score[i] = 1.0

# LTokenizer
tokenizer = LTokenizer(scores=cohesion_score)

all cohesion probabilities was computed. # words = 1578
all branching entropies was computed # words = 9138
all accessor variety was computed # words = 9138


In [ ]:
# vectorize new sentences based on doc2vec
input_vec = model_soynlp.infer_vector(tokenizer.tokenize(input_sent))

# most similarity top 5
return_docs = model_soynlp.docvecs.most_similar(positive=[input_vec], topn=5)

# find Similar Sentences in the Trained Dataframe
result = [df.u.iloc[int(i[0])] for i in return_docs]

# result
if df_ref.empty:
    print("유사 문장이 없습니다.")
else:
    print(f"입력 문장 : {input_sent}")
    print("-"*100)
    for k, i in enumerate(zip(return_docs, result)):
        print(f"{k+1}.")
        print(f"유사 문장 : {i[1]['u']}")
        print(f"유사도 : {i[0][1]}")

        # find Similar Sentences in Dialogueset
        temp_ref = df_ref[df_ref.u.values == i[1]['u']]
        
        temp_ref = temp_ref.sample(frac=1).reset_index(drop=True)
        result = str(temp_ref.meme.iloc[0])
        print(f"밈 답변 : {result.strip()}")
        print("-"*100)

입력 문장 : 오늘 날씨가 너무 좋다
----------------------------------------------------------------------------------------------------
1.
유사 문장 : 오늘 날씨가 너무 쾌청해!!
유사도 : 0.7561725378036499
밈 답변 : meme    후후 그거 다 내 덕분임 네가 말로만 듣던 걔가 나야↗ 날씨요정♬
Name: 0, dtype: object
----------------------------------------------------------------------------------------------------
2.
유사 문장 : 웹툰 진심 너무 웃겨
유사도 : 0.7292321920394897
밈 답변 : meme    한 대기리가 있어... 웹툰을 너무 사랑한...
Name: 0, dtype: object
----------------------------------------------------------------------------------------------------
3.
유사 문장 : 나 오늘 너무 예쁘다
유사도 : 0.6951525211334229
밈 답변 : meme    이 높은 자존감 뭐지? 대길이 선 넘네
Name: 0, dtype: object
----------------------------------------------------------------------------------------------------
4.
유사 문장 : 날씨가 진짜 너무 좋은데!
유사도 : 0.6713443994522095
밈 답변 : meme    내 덕분인 거 아니? 네가 말로만 듣던 걔가 나야↗ 날씨요정~!
Name: 0, dtype: object
----------------------------------------------------------------------------------------------------
5